In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder,StandardScaler
from tqdm import tqdm
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
import json
import xgboost as xgb
from pathlib import Path
import torch
import json
import numpy as np
import pickle
import spacy_transformers

In [8]:
df_train = pd.read_json('/kaggle/input/fi-da-taset-1/movie_training.json', lines=False)
df_train

,text,intent,entities
0,phim hành động đánh giá trên trung bình giống ...,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'trên tru..."
1,phim hài hước đánh giá cao giống trận đánh khô...,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'cao', 's..."
2,hôm nay xem gì đó,khac,[]
3,phim tâm lý đánh giá tốt giống muốn cả thế giớ...,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'tốt', 's..."
4,tôi muốn xem phim có bé ma của anh my boo 2024...,tim_kiem_phim,"[{'entity': 'keywords', 'value': 'bé ma của an..."
...,...,...,...
62995,phim của đạo diễn han jun-hee,goi_y_theo_dao_dien,"[{'entity': 'director', 'value': 'han jun-hee'..."
62996,phim tình cảm năm 1943 giốn,khac,[]
62997,phim nào của yo hitoto hay,goi_y_theo_dien_vien,"[{'entity': 'actor', 'value': 'yo hitoto', 'st..."
62998,phim nào của alton yu hay,goi_y_theo_dien_vien,"[{'entity': 'actor', 'value': 'alton yu', 'sta..."


In [9]:
df_valid = pd.read_json('/kaggle/input/fi-da-taset-1/movie_validation.json', lines=False)
df_valid

,text,intent,entities
0,tìm phim tâm lý năm 1982,goi_y_theo_nam,"[{'entity': 'year', 'value': '1982', 'start': ..."
1,so sánh gặp em ở manhattan và điệu nhảy khuy...,so_sanh_phim,"[{'entity': 'title1', 'value': 'gặp em ở man..."
2,phim phiêu lưu đánh giá trên trung bình giống ...,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'trên tru..."
3,phim nào của toosafe krittawat suwanich hay,goi_y_theo_dien_vien,"[{'entity': 'actor', 'value': 'toosafe krittaw..."
4,so sánh kamen rider wizard &amp; fourze movie ...,so_sanh_phim,"[{'entity': 'title1', 'value': 'kamen rider wi..."
...,...,...,...
17995,phim phiêu lưu của edmund yeo,goi_y_ket_hop,"[{'entity': 'genre', 'value': 'phiêu lưu', 'st..."
17996,gợi ý phim hành động như linh hồn bạc 2: luật ...,goi_y_theo_the_loai,"[{'entity': 'genre', 'value': 'hành động', 'st..."
17997,phim viễn tưởng đánh giá tốt giống who kissed me,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'tốt', 's..."
17998,phim tình cảm đánh giá cao giống chiến hữu,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'cao', 's..."


In [10]:
df_test = pd.read_json('/kaggle/input/fi-da-taset-1/movie_test.json', lines=False)
df_test

,text,intent,entities
0,tìm phim hành động năm 1967,goi_y_theo_nam,"[{'entity': 'year', 'value': '1967', 'start': ..."
1,phim nào của raita ryû hay,goi_y_theo_dien_vien,"[{'entity': 'actor', 'value': 'raita ryû', 'st..."
2,phim tâm lý của tôn hạo,goi_y_ket_hop,"[{'entity': 'genre', 'value': 'tâm lý', 'start..."
3,giá vàng điện thoại thể thao giá vàng chứng kh...,khac,[]
4,tìm phim về witness 1985 đây là một trong nhữn...,tim_kiem_phim,"[{'entity': 'keywords', 'value': 'witness 1985..."
...,...,...,...
8995,phim của đạo diễn ken*ichi takeshita,goi_y_theo_dao_dien,"[{'entity': 'director', 'value': 'ken*ichi tak..."
8996,phim của đạo diễn bong joon-ho,goi_y_theo_dao_dien,"[{'entity': 'director', 'value': 'bong joon-ho..."
8997,phim võ thuật đánh giá cao giống thợ săn ma túy,goi_y_theo_danh_gia,"[{'entity': 'rating_level', 'value': 'cao', 's..."
8998,tìm phim về waen dok mai 2017 waen dok mai 207...,tim_kiem_phim,"[{'entity': 'keywords', 'value': 'waen dok mai..."


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thiết bị: {device}")

# Load mô hình SimCSE
simcse_model_name = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
tokenizer = AutoTokenizer.from_pretrained(simcse_model_name)
simcse_model = AutoModel.from_pretrained(simcse_model_name).to(device)

# Hàm trích xuất embedding
def get_embedding_batched(texts, tokenizer, model, batch_size, max_length, device):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Creating Embeddings"):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embedding = normalize(embedding, norm="l2")  # chuẩn hóa nếu muốn
            all_embeddings.append(embedding)
    return np.vstack(all_embeddings)

df_train_data = pd.read_json('/kaggle/input/fi-da-taset-1/movie_training.json', lines=False)
X_train = df_train_data['text'].tolist()
y_train = df_train_data['intent'].tolist()

df_valid_data = pd.read_json('/kaggle/input/fi-da-taset-1/movie_validation.json', lines=False)
X_valid = df_valid_data['text'].tolist()
y_valid = df_valid_data['intent'].tolist()

df_test_data = pd.read_json('/kaggle/input/fi-da-taset-1/movie_test.json', lines=False)
X_test = df_test_data['text'].tolist()
y_test = df_test_data['intent'].tolist()

batch_size = 256  # Điều chỉnh kích thước batch tùy theo bộ nhớ của bạn
max_length = 256 
X_train_embeddings = get_embedding_batched(X_train, tokenizer, simcse_model, batch_size, max_length, device)
X_valid_embeddings = get_embedding_batched(X_valid, tokenizer, simcse_model, batch_size, max_length, device)
X_test_embeddings = get_embedding_batched(X_test, tokenizer, simcse_model, batch_size, max_length, device)

# Encode nhãn
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)
y_test_encoded = label_encoder.transform(y_test)



Thiết bị: cuda


2025-04-15 06:41:08.466265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744699268.660504      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744699268.714279      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Creating Embeddings: 100%|██████████| 36/36 [01:56<00:00,  3.23s/it]


In [11]:
# Chuẩn hóa embedding
scaler = StandardScaler()
X_train_embeddings_scaled = scaler.fit_transform(X_train_embeddings)
X_valid_embeddings_scaled = scaler.transform(X_valid_embeddings)
X_test_embeddings_scaled = scaler.transform(X_test_embeddings)

In [12]:
xgb_model = xgb.XGBClassifier(
    device='cuda',
    use_label_encoder=False,
    eval_metric=['mlogloss', 'merror'],
    max_depth=5,  
    learning_rate=0.02, 
    n_estimators=1000,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    early_stopping_rounds=20
)

xgb_model.fit(
    X_train_embeddings_scaled,
    y_train_encoded,
    eval_set=[(X_valid_embeddings_scaled, y_valid_encoded)],
    verbose=True
)

[0]	validation_0-mlogloss:2.14638	validation_0-merror:0.23300
[1]	validation_0-mlogloss:2.09823	validation_0-merror:0.17572
[2]	validation_0-mlogloss:2.05174	validation_0-merror:0.14861
[3]	validation_0-mlogloss:2.00779	validation_0-merror:0.13483
[4]	validation_0-mlogloss:1.96668	validation_0-merror:0.12833
[5]	validation_0-mlogloss:1.92716	validation_0-merror:0.12456
[6]	validation_0-mlogloss:1.88989	validation_0-merror:0.12256
[7]	validation_0-mlogloss:1.85440	validation_0-merror:0.11989
[8]	validation_0-mlogloss:1.82015	validation_0-merror:0.11778
[9]	validation_0-mlogloss:1.78741	validation_0-merror:0.11794
[10]	validation_0-mlogloss:1.75562	validation_0-merror:0.11833
[11]	validation_0-mlogloss:1.72513	validation_0-merror:0.11783
[12]	validation_0-mlogloss:1.69546	validation_0-merror:0.11556
[13]	validation_0-mlogloss:1.66723	validation_0-merror:0.11394
[14]	validation_0-mlogloss:1.63982	validation_0-merror:0.11194
[15]	validation_0-mlogloss:1.61320	validation_0-merror:0.11156
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device='cuda', early_stopping_rounds=20,
              enable_categorical=False, eval_metric=['mlogloss', 'merror'],
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [13]:
from sklearn.metrics import classification_report, accuracy_score


y_pred_test = xgb_model.predict(X_test_embeddings_scaled)

accuracy = accuracy_score(y_test_encoded, y_pred_test)
print(f"Độ chính xác trên tập test: {accuracy:.2f}")

y_pred_test_labels = label_encoder.inverse_transform(y_pred_test)
y_test_data_labels = label_encoder.inverse_transform(y_test_encoded)

print("Báo cáo phân loại:\n", classification_report(y_test_data_labels, y_pred_test_labels))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [07:04:09] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Độ chính xác trên tập test: 0.97
Báo cáo phân loại:
                       precision    recall  f1-score   support

       goi_y_ket_hop       0.97      0.99      0.98      1000
 goi_y_theo_danh_gia       0.98      0.99      0.99      1000
 goi_y_theo_dao_dien       0.99      1.00      0.99      1000
goi_y_theo_dien_vien       0.98      0.99      0.99      1000
      goi_y_theo_nam       0.98      1.00      0.99      1000
 goi_y_theo_the_loai       0.97      0.98      0.98      1000
                khac       0.96      0.82      0.89      1000
        so_sanh_phim       0.96      0.99      0.97      1000
       tim_kiem_phim       0.95      0.99      0.97      1000

            accuracy                           0.97      9000
           macro avg       0.97      0.97      0.97      9000
        weighted avg       0.97      0.97      0.97      9000



In [14]:
import joblib

xgb_model.save_model('/kaggle/working/intent_recognition_model.json')
with open('/kaggle/working/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
with open('/kaggle/working/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

### Mô hình dự đoán thực thể

In [15]:
def remove_overlapping_entities(entities):
    entities = sorted(entities, key=lambda x: x[0])
    result = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:
            result.append((start, end, label))
            last_end = end
    return result


def load_data(data, lang="vi"):
    nlp = spacy.blank(lang)
    examples = []
    for item in data:
        text = item["text"]
        doc = nlp.make_doc(text)
        ents = []
        for ent in item["entities"]:
            start, end, label = ent["start"], ent["end"], ent["entity"]
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append((start, end, label))
        ents = remove_overlapping_entities(ents)
        example = Example.from_dict(doc, {"entities": ents})
        examples.append(example)
    return examples


def create_or_load_model(model_path=None):
    if model_path and Path(model_path).exists():
        print(f"Đang load mô hình đã huấn luyện từ: {model_path}")
        nlp = spacy.load(model_path)
        return nlp, nlp.get_pipe("ner"), True
    else:
        print("Tạo mô hình mới từ đầu...")
        nlp = spacy.blank("vi")
        transformer_config = {
            "model": {
                "@architectures": "spacy-transformers.TransformerModel.v3",
                "name": "NlpHUST/ner-vietnamese-electra-base",
                "tokenizer_config": {"use_fast": True},
                "transformer_config": {},
                "mixed_precision": False,
                "grad_scaler_config": {},
                "get_spans": {
                    "@span_getters": "spacy-transformers.strided_spans.v1",
                    "window": 128,
                    "stride": 96
                }
            },
            "set_extra_annotations": {
                "@annotation_setters": "spacy-transformers.null_annotation_setter.v1"
            }
        }
        nlp.add_pipe("transformer", config=transformer_config, first=True)
        ner = nlp.add_pipe("ner", last=True)
        nlp.initialize()
        return nlp, ner, False


def freeze_transformer_layers(nlp, freeze=True):
    transformer_pipe = nlp.get_pipe("transformer")
    if freeze:
        # Chỉ freeze các tham số của mô hình transformer (không freeze tokenizer)
        for name, param in transformer_pipe.model.transformer.named_parameters():
            param.requires_grad = False
        print("Các layer transformer đã được freeze.")


def train_model(nlp, ner, train_data, val_data, output_dir,
                is_existing_model=False, n_iter=15, batch_size=128, dropout=0.1,
                learning_rate=1e-5, freeze_transformer_flag=True):
    # Thêm nhãn dựa trên dữ liệu huấn luyện
    for example in train_data:
        for ent in example.reference.ents:
            if ent.label_ not in ner.labels:
                ner.add_label(ent.label_)

    # Freeze transformer
    if freeze_transformer_flag:
        freeze_transformer_layers(nlp, freeze=True)

    other_pipes = [p for p in nlp.pipe_names if p != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.initialize(lambda: train_data) if not is_existing_model else nlp.resume_training()

        # Điều chỉnh learning rate
        if hasattr(optimizer, "param_groups"):  # Kiểm tra nếu optimizer có param_groups
            for param_group in optimizer.param_groups:
                param_group["lr"] = learning_rate

        best_f = 0.0
        patience = 5
        patience_counter = 0
        unfrozen = False  # Đánh dấu bỏ freeze transformer

        for epoch in range(n_iter):
            print(f"\nEpoch {epoch + 1}/{n_iter}")
            random.shuffle(train_data)
            losses = {}
            batch_losses = []

            batches = minibatch(train_data, size=batch_size)
            progress = tqdm(batches,
                            total=(len(train_data) + batch_size - 1) // batch_size,
                            desc=f"Epoch {epoch+1}/{n_iter}")

            for batch in progress:
                nlp.update(batch, sgd=optimizer, drop=dropout, losses=losses)
                if "ner" in losses:
                    batch_losses.append(losses["ner"])
                    progress.set_postfix(loss=losses["ner"])

            avg_loss = sum(batch_losses) / len(batch_losses) if batch_losses else 0
            scorer = nlp.evaluate(val_data)
            f_score = scorer["ents_f"]

            print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, F1 = {f_score:.4f}")

            if f_score > best_f:
                best_f = f_score
                patience_counter = 0
                nlp.to_disk(Path(output_dir))
                print(f"Mô hình tốt nhất đã được lưu tại: {output_dir}")
            else:
                patience_counter += 1
                print(f"Không cải thiện. Patience hiện tại: {patience_counter}/{patience}")
                
                # Sau 5 epoch đầu nếu chưa cải thiện tốt, bỏ freeze transformer
                if patience_counter == 3 and freeze_transformer_flag and not unfrozen:
                    freeze_transformer_layers(nlp, freeze=False)
                    unfrozen = True
                    print("Transformer đã được bỏ khóa để fine-tune sâu hơn.")

                if patience_counter >= patience:
                    print("Dừng huấn luyện sớm do không cải thiện.")
                    break

    return nlp

In [16]:
# Load dữ liệu
with open('/kaggle/input/fi-da-taset-1/movie_training.json', "r", encoding="utf-8") as f:
    data_train = json.load(f)
with open('/kaggle/input/fi-da-taset-1/movie_validation.json', "r", encoding="utf-8") as f:
    data_valid = json.load(f)
with open('/kaggle/input/fi-da-taset-1/movie_test.json', "r", encoding="utf-8") as f:
    data_test = json.load(f)

train_examples = load_data(data_train, lang="vi")
valid_examples = load_data(data_valid, lang="vi")
test_examples = load_data(data_test, lang="vi")

In [17]:
spacy.prefer_gpu()
model_path = "/kaggle/working/model_ner_NVEB"

nlp, ner, is_existing = create_or_load_model(model_path)
trained_nlp = train_model(
    nlp, ner,
    train_data=train_examples,
    val_data=valid_examples,
    output_dir=model_path,
    is_existing_model=is_existing,
    n_iter=15,
    batch_size=128,
    dropout=0.1,
    learning_rate=1e-5,
    freeze_transformer_flag=True
)


[2025-04-15 07:05:02,739] [INFO] Created vocabulary
[2025-04-15 07:05:02,739] [INFO] Finished initializing nlp object


Tạo mô hình mới từ đầu...


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

...

Epoch 13: Loss = 989.0435, F1 = 0.9892
Không cải thiện. Patience hiện tại: 3/5

Epoch 14/15


Epoch 14/15: 100%|██████████| 493/493 [02:11<00:00,  3.76it/s, loss=1.96e+3]


Epoch 14: Loss = 914.4608, F1 = 0.9897
Không cải thiện. Patience hiện tại: 4/5

Epoch 15/15


Epoch 15/15: 100%|██████████| 493/493 [02:15<00:00,  3.64it/s, loss=1.85e+3]


Epoch 15: Loss = 885.7969, F1 = 0.9907
Mô hình tốt nhất đã được lưu tại: /kaggle/working/model_ner_NVEB


In [18]:
nlp = spacy.load("/kaggle/working/model_ner_NVEB")
scorer_test = nlp.evaluate(test_examples)
print(f"Evaluate test data: ")
print(scorer_test["ents_f"])

/usr/local/lib/python3.11/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fileli

Evaluate test data: 
0.991073454875558
